In [1]:
import sys
import os

REPO_PATH = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if REPO_PATH not in sys.path:
    sys.path.insert(0, REPO_PATH)

    
from llama_cpp import Llama
from src.prompts.prompts import SYSTEM_PROMPT, SYSTEM_PROMPT_CLASSIFICATION
from src.app.llm import generate_response
from src.app.classif import generate_response_classification

In [2]:
MODELS_CONFIG = {
    "mistral-7b": { # 4.3Go
        "path": "../models/mistral-7b-instruct-v0.2.Q4_K_M.gguf", # C'est la version conçue pour le chat et les instructions
    },
    "qwen2.5-1.5b": { # 1.6Go
        "path": "../models/qwen2.5-1.5b-instruct-q8_0.gguf", # Excellente performance, en particulier sur les langues non-anglaises.
    },
    "qwen2.5-7b": { # 1.6Go
        "path": "../models/qwen2.5-7b-instruct-q2_k.gguf", # Excellente performance, en particulier sur les langues non-anglaises.
    }
}

MODEL_PATH = MODELS_CONFIG["mistral-7b"]["path"]
MODEL_PATH = MODELS_CONFIG["qwen2.5-1.5b"]["path"]
MODEL_PATH = MODELS_CONFIG["qwen2.5-7b"]["path"]

import sys
import os
from contextlib import contextmanager

@contextmanager
def suppress_llama_output():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr

with suppress_llama_output():
    llm = Llama(
        model_path=MODEL_PATH,
        n_ctx=2048,
        chat_format="chatml",
        verbose=False
    )


In [3]:
def generate_response(llm, question):
    """
    Fonction qui envoie la phrase de l'utilisateur au LLM et traite le résultat.
    Utilise une approche "few-shot" avec 4 exemples diversifiés pour guider le modèle.
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": question}
    ]

    response = llm.create_chat_completion(
        messages=messages,
        temperature=0.3,
        max_tokens=512
    )

    return response['choices'][0]['message']['content'].strip()

def generate_response_classification(llm, question):
    """
    Fonction qui envoie la phrase de l'utilisateur au LLM et traite le résultat.
    Utilise une approche "few-shot" avec 4 exemples diversifiés pour guider le modèle.
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT_CLASSIFICATION},
        {"role": "user", "content": question}
    ]

    response = llm.create_chat_completion(
        messages=messages,
        temperature=0,
        max_tokens=10
    )

    return response['choices'][0]['message']['content'].strip()

In [4]:
list_questions_grave = ['J’ai mal à la tête depuis deux jours, est-ce inquiétant ?', 'J’ai une douleur thoracique intense qui irradie dans le bras gauche', 'J’ai de la fièvre à 38°C et le nez qui coule', 'Je n’arrive plus à respirer correctement depuis ce matin', 'J’ai des vertiges quand je me lève trop vite', 'Mon enfant fait une convulsion avec de la fièvre', 'J’ai une toux sèche depuis une semaine', 'Je vomis du sang depuis ce matin', 'J’ai une petite coupure au doigt', 'J’ai une perte soudaine de la vision d’un œil', 'J’ai mal au ventre après avoir mangé', 'J’ai un engourdissement soudain du visage et du bras droit', 'J’ai des courbatures et de la fatigue', 'Mon cœur bat très vite et je me sens faible', 'J’ai une éruption cutanée qui me gratte']
    
for question in list_questions_grave:
    reponse = generate_response(llm, question)
    proba = float(generate_response_classification(llm, question))
    if proba >= 0.5:
        print(f"Question : {question}\nRéponse : {reponse + '\nSouhaitez-vous passer en téléconsultation maintenant ?'}\nProba : {proba}\n")
    else:
        print(f"Question : {question}\nRéponse : {reponse + '\nAu vu de vos symptômes, il n’est pas nécessaire de programmer une téléconsultation immédiatement.'}\nProba : {proba}\n")

Question : J’ai mal à la tête depuis deux jours, est-ce inquiétant ?
Réponse : Il est normal de se soucier quand on a mal à la tête, mais il est important de comprendre que les informations fournies ici sont de simples informations générales et non un avis médical. Voici quelques points à considérer :

1. La plupart des maux de tête sont temporaires et non alarmants. Ils peuvent être causés par la fatigue, la déshydratation, le stress ou certaines activités physiques.

2. Si vous avez des douleurs de tête qui sont récurrentes ou s'accompagnent de symptômes tels que la nausée, la vomissements, la sensibilité à la lumière ou la sensibilité au bruit, cela pourrait indiquer un type de maux de tête qui nécessite une attention supplémentaire.

3. Il est toujours bon de bien hydrater, de bien dormir et de maintenir un régime alimentaire équilibré pour aider à prévenir les maux de tête.

Si vous vous inquiétez ou si les maux de tête persistent, il est toujours conseillé de consulter un profess

# Evaluation

In [12]:
MODELS_CONFIG = {
    "mistral-7b": { # 4.3Go
        "path": "../models/mistral-7b-instruct-v0.2.Q4_K_M.gguf", # C'est la version conçue pour le chat et les instructions
    },
    "qwen2.5-1.5b": { # 1.6Go
        "path": "../models/qwen2.5-1.5b-instruct-q8_0.gguf", # Excellente performance, en particulier sur les langues non-anglaises.
    },
    "qwen2.5-7b": { # 1.6Go
        "path": "../models/qwen2.5-7b-instruct-q2_k.gguf", # Excellente performance, en particulier sur les langues non-anglaises.
    }
}

list_models = [MODELS_CONFIG["mistral-7b"]["path"], MODELS_CONFIG["qwen2.5-1.5b"]["path"], MODELS_CONFIG["qwen2.5-7b"]["path"]]

import sys
import os
from contextlib import contextmanager

@contextmanager
def suppress_llama_output():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr


In [ ]:
import json

# Ouvrir le fichier JSON
with open('../data/data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Afficher les données
label = [item['grave'] for item in data]
question = [item['question_patient'] for item in data]
print(label)
print(question)

[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
['J’ai mal à la tête depuis deux jours, est-ce inquiétant ?', 'J’ai une douleur thoracique intense qui irradie dans le bras gauche', 'J’ai de la fièvre à 38°C et le nez qui coule', 'Je n’arrive plus à respirer correctement depuis ce matin', 'J’ai des vertiges quand je me lève trop vite', 'Mon enfant fait une convulsion avec de la fièvre', 'J’ai une toux sèche depuis une semaine', 'Je vomis du sang depuis ce matin', 'J’ai une petite coupure au doigt', 'J’ai une perte soudaine de la vision d’un œil', 'J’ai mal au ventre après avoir mangé', 'J’ai un engourdissement soudain du visage et du bras droit', 'J’ai des courbatures et de la fatigue', 'Mon cœur bat très vite et je me sens faible', 'J’ai une éruption cutanée qui me gratte']


In [14]:
list_evaluation = []
for MODEL_PATH in list_models:
    list_proba = []
    print(f"Using model: {MODEL_PATH}")
    for q in question:
        with suppress_llama_output():
            llm = Llama(
                model_path=MODEL_PATH,
                n_ctx=2048,
                chat_format="chatml",
                verbose=False
            )
        proba = generate_response_classification(q)
        list_proba.append(proba)
    print(list_proba)
    list_evaluation.append(list_proba)

Using model: ../models/mistral-7b-instruct-v0.2.Q4_K_M.gguf
['0.3', '0.8\n<|im_end|', '0.5\n<|im_end|', '0.8\n<|im_end|', '0.3', '0.8\n<|im_end|', '0.3\n<|im_end|', '0.5', '0.2\n<|im_end|', '1', '0.2\n<|im_end|', '0.5\n<|im_end|', '0.3\n<|im_end|', '0.6\n<|im_end|', '0.5\n<|im_end|']
Using model: ../models/qwen2.5-1.5b-instruct-q8_0.gguf
['0', '1', '0.8', '1', '0.8', '1', '0', '1', '0', '1', '0', '0', '0', '0.9', '0']
Using model: ../models/qwen2.5-7b-instruct-q2_k.gguf
['0.3', '0.95', '0.3', '0.95', '0.1', '0.95', '0.1', '0.95', '0', '0.95', '0.1', '0.8', '0.1', '0.8', '0.2']


In [18]:
from sklearn.metrics import confusion_matrix



for liste in list_evaluation:
    y_pred = []
    liste_nettoyee = [x.replace('\n<|im_end|', '').strip() for x in liste]
    liste_nettoyee = [float(x) for x in liste_nettoyee]
    
    for n in liste_nettoyee:
        if n < 0.5:
            n=0
        if n >= 0.5:
            n=1
        y_pred.append(n)
        
    y_true = label
    cm = confusion_matrix(y_true, y_pred)

    print("Matrice de confusion :")
    print(cm)

Matrice de confusion :
[[6 2]
 [0 7]]
Matrice de confusion :
[[6 2]
 [1 6]]
Matrice de confusion :
[[8 0]
 [0 7]]
